In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout, Reshape

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle


In [4]:
path = 'C:/Users/lyc07/Desktop/Columbia/S3/GR5243 Applied Data Science/prj3/'
dat_full = pd.read_csv(path + 'dat_full.csv')
dat_full.drop('Unnamed: 0', inplace=True, axis=1)
dat_full.drop('Unnamed: 0.1', inplace=True, axis=1)

In [5]:
dat_full.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature5998,feature5999,feature6000,feature6001,feature6002,feature6003,feature6004,feature6005,feature6006,emotion_idx
0,31,18,1,20,33,20,1,17,151,116,...,98,150,202,50,102,154,52,104,52,21
1,35,15,6,27,39,20,2,17,151,151,...,104,159,213,54,109,163,55,109,54,11
2,35,17,2,20,32,14,3,20,165,122,...,99,152,204,52,105,157,53,105,52,2
3,33,16,3,20,29,12,6,23,161,133,...,101,156,210,54,109,163,55,109,54,7
4,29,15,1,17,27,13,2,17,133,102,...,83,128,172,44,89,133,45,89,44,1


In [6]:
X, y = dat_full.iloc[:,:-1],dat_full.iloc[:,-1]
y = y-1

In [7]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99, whiten=True)
X_pca = pca.fit_transform(X)

In [8]:
#from sklearn.model_selection import train_test_split
#import keras

X_train, X_test, y_train, y_test = train_test_split(X_pca, np.asarray(y), test_size=0.2, random_state = 123)

# The known number of output classes.
num_classes = 22

# Input image dimensions
input_shape = (4,)

# Convert class vectors to binary class matrices. This uses 1 hot encoding.
#y_train_binary = to_categorical(y_train, num_classes)
#y_test_binary = to_categorical(y_test, num_classes)

X_train = X_train.reshape(2000, 92,1)
X_test = X_test.reshape(500, 92,1)

In [18]:
from tensorflow.keras.regularizers import l1, l2

In [21]:
model = Sequential()
model.add(Conv1D(800, (1), input_shape=(92,1), activation='relu',activity_regularizer=l2(0.001)))
model.add(Conv1D(600,(1), activation = 'relu'))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(600, activation='relu',activity_regularizer=l1(0.001)))
model.add(Dense(300, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(loss= 'sparse_categorical_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_8 (Conv1D)            (None, 92, 800)           1600      
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 92, 600)           480600    
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 46, 600)           0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 27600)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 600)               16560600  
_________________________________________________________________
dense_17 (Dense)             (None, 300)               180300    
_________________________________________________________________
dense_18 (Dense)             (None, 200)              

In [22]:
epochs = 20
model.fit(X_train, y_train,
          epochs=epochs,
          verbose=1,
          validation_split = 0.2)

Train on 1600 samples, validate on 400 samples
Epoch 1/20
1600/1600 [==============================] - 7s 4ms/sample - loss: 3.1260 - accuracy: 0.0512 - val_loss: 3.0955 - val_accuracy: 0.0550
Epoch 2/20
1600/1600 [==============================] - 6s 4ms/sample - loss: 3.0894 - accuracy: 0.0600 - val_loss: 3.0839 - val_accuracy: 0.0675
Epoch 3/20
1600/1600 [==============================] - 6s 4ms/sample - loss: 3.0075 - accuracy: 0.1106 - val_loss: 2.8845 - val_accuracy: 0.1225
Epoch 4/20
1600/1600 [==============================] - 6s 4ms/sample - loss: 2.7163 - accuracy: 0.1719 - val_loss: 2.6698 - val_accuracy: 0.1725
Epoch 5/20
1600/1600 [==============================] - 6s 4ms/sample - loss: 2.4302 - accuracy: 0.2625 - val_loss: 2.6342 - val_accuracy: 0.1900
Epoch 6/20
1600/1600 [==============================] - 6s 4ms/sample - loss: 2.1266 - accuracy: 0.3300 - val_loss: 2.6808 - val_accuracy: 0.2125
Epoch 7/20
1600/1600 [==============================] - 6s 4ms/sample - loss:

In [11]:
pred = model.predict(X_test)

In [12]:
pred_index = np.argmax(pred, axis = 1)
accuracy = accuracy_score(y_test, pred_index)
accuracy

0.264